# Imports & Load Previous Models

We reuse:

- the **PreCNN encoder**, which maps each CSI window to
  an embedding vector $z_{\text{pre}} \in \mathbb{R}^{128}$,
- the **fusion model**, which takes $(z_{\text{pre}}, z_{\text{phys}})$
  and outputs:

  - presence logits / probabilities for the binary label
    $y_{\text{cls}} \in \{0,1\}$ (empty vs non-empty),
  - coordinates $\hat{\mathbf{p}} = (\hat{x}, \hat{y})$ for localization.

In this notebook we fine-tune the fusion model in two phases while
reusing the saved weights from previous steps.

In [1]:
# =========================================================
# CELL 1 — Imports, Paths, Load Models
# =========================================================
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import optimizers, losses, callbacks

print("TensorFlow:", tf.__version__)

MODEL_DIR = "/home/tonyliao/Location_new_aoa_PDF/models"
os.makedirs(MODEL_DIR, exist_ok=True)

ENC_SAVE_PATH   = f"{MODEL_DIR}/prec_encoder.h5"
FULL_MODEL_PATH = f"{MODEL_DIR}/loc_full_model.h5"

# Load without compile; we will re-compile below
encoder = tf.keras.models.load_model(ENC_SAVE_PATH, compile=False)
full_model = tf.keras.models.load_model(FULL_MODEL_PATH, compile=False)

def euclid_mean(y_true, y_pred):
    return tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.square(y_true - y_pred), axis=-1)))

# Helper: recursively find a submodel whose name contains keyword
def find_submodel(root_model, keyword: str):
    keyword = keyword.lower()
    for layer in root_model.layers:
        if isinstance(layer, tf.keras.Model) and keyword in layer.name.lower():
            return layer
        if isinstance(layer, tf.keras.Model):
            hit = find_submodel(layer, keyword)
            if hit is not None:
                return hit
    return None

# Helper: set trainable recursively
def set_trainable_recursive(root_model, trainable: bool, name_contains: str = None):
    key = name_contains.lower() if name_contains else None
    for layer in root_model.layers:
        if (key is None) or (key in layer.name.lower()):
            layer.trainable = trainable
        if isinstance(layer, tf.keras.Model):
            set_trainable_recursive(layer, trainable, name_contains)

print("Loaded:")
print("  encoder   =", ENC_SAVE_PATH)
print("  full_model=", FULL_MODEL_PATH)

2026-01-15 19:09:04.711266: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-15 19:09:04.717507: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768504144.724681 2340938 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768504144.726885 2340938 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768504144.732486 2340938 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

TensorFlow: 2.19.0


I0000 00:00:1768504145.950815 2340938 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22181 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9


Loaded:
  encoder   = /home/tonyliao/Location_new_aoa_PDF/models/prec_encoder.h5
  full_model= /home/tonyliao/Location_new_aoa_PDF/models/loc_full_model.h5


In [2]:
# =========================================================
# CELL 2 — Load Step3-5 Arrays + Train/Val Split
# =========================================================
Z_PRE_PATH   = "/home/tonyliao/Location_new_aoa_PDF/Z_PRE.npy"
Z_PHYS_PATH  = "/home/tonyliao/Location_new_aoa_PDF/Z_PHYS.npy"
YCLS_PATH    = "/home/tonyliao/Location_new_aoa_PDF/Y_cls.npy"
YLOC_PATH    = "/home/tonyliao/Location_new_aoa_PDF/coords.npy"

Z_PRE  = np.load(Z_PRE_PATH)
Z_PHYS = np.load(Z_PHYS_PATH)
Y_cls  = np.load(YCLS_PATH)
Y_loc  = np.load(YLOC_PATH)

print("Loaded arrays:")
print("  Z_PRE :", Z_PRE.shape)
print("  Z_PHYS:", Z_PHYS.shape)
print("  Y_cls :", Y_cls.shape, Y_cls.dtype)
print("  Y_loc :", Y_loc.shape, Y_loc.dtype)

N = len(Y_cls)
rng = np.random.default_rng(42)
idx = np.arange(N)
rng.shuffle(idx)

split = int(N * 0.8)
train_idx, val_idx = idx[:split], idx[split:]

Z_PRE_train,  Z_PRE_val  = Z_PRE[train_idx],  Z_PRE[val_idx]
Z_PHYS_train, Z_PHYS_val = Z_PHYS[train_idx], Z_PHYS[val_idx]
Y_cls_train,  Y_cls_val  = Y_cls[train_idx],  Y_cls[val_idx]
Y_loc_train,  Y_loc_val  = Y_loc[train_idx],  Y_loc[val_idx]

print("Split:")
print("  Train:", Z_PRE_train.shape, Z_PHYS_train.shape, Y_cls_train.shape, Y_loc_train.shape)
print("  Val  :", Z_PRE_val.shape,  Z_PHYS_val.shape,  Y_cls_val.shape,  Y_loc_val.shape)

Loaded arrays:
  Z_PRE : (166149, 128)
  Z_PHYS: (166149, 5)
  Y_cls : (166149,) int32
  Y_loc : (166149, 2) float32
Split:
  Train: (132919, 128) (132919, 5) (132919,) (132919, 2)
  Val  : (33230, 128) (33230, 5) (33230,) (33230, 2)


In [3]:
# =========================================================
# CELL 3 — Coordinate Normalize + Save coord_mean/std
# =========================================================
coord_mean = Y_loc_train.mean(axis=0).astype(np.float32)
coord_std  = (Y_loc_train.std(axis=0) + 1e-6).astype(np.float32)

np.save(f"{MODEL_DIR}/coord_mean.npy", coord_mean)
np.save(f"{MODEL_DIR}/coord_std.npy",  coord_std)

Y_loc_train_n = ((Y_loc_train - coord_mean) / coord_std).astype(np.float32)
Y_loc_val_n   = ((Y_loc_val   - coord_mean) / coord_std).astype(np.float32)

print("coord_mean:", coord_mean)
print("coord_std :", coord_std)
print("Saved coord_mean/std →", MODEL_DIR)

coord_mean: [2.4951699 2.0143394]
coord_std : [1.6937784 1.6303215]
Saved coord_mean/std → /home/tonyliao/Location_new_aoa_PDF/models


In [4]:
# =========================================================
# CELL 4 — Phase 1: Freeze PreCNN + Compile + Train
# =========================================================
# Freeze encoder copy (optional)
for layer in encoder.layers:
    layer.trainable = False

# Freeze the PreCNN inside full_model (robust)
precnn = find_submodel(full_model, "precnn") or find_submodel(full_model, "PreCNN")
if precnn is not None:
    set_trainable_recursive(precnn, False)
    print("Phase 1: froze submodel:", precnn.name)
else:
    # fallback: freeze layers whose name contains 'precnn'
    set_trainable_recursive(full_model, False, name_contains="precnn")
    print("Phase 1: fallback freeze by name_contains='precnn'")

LR_PHASE1 = 1e-3
full_model.compile(
    optimizer=optimizers.Adam(LR_PHASE1),
    loss={
        "presence": losses.SparseCategoricalCrossentropy(),
        "coords":   losses.MeanSquaredError(),
    },
    loss_weights={"presence": 0.2, "coords": 1.0},
    metrics={"presence": ["accuracy"], "coords": [euclid_mean]},
)

ckpt_phase1 = callbacks.ModelCheckpoint(
    f"{MODEL_DIR}/fine_phase1.h5",
    save_best_only=True,
    monitor="val_coords_loss",
    mode="min",
    verbose=1,
)
es_phase1 = callbacks.EarlyStopping(
    monitor="val_coords_loss",
    mode="min",
    patience=10,
    restore_best_weights=True,
    verbose=1,
)

history_phase1 = full_model.fit(
    x=[Z_PRE_train, Z_PHYS_train],
    y={"presence": Y_cls_train, "coords": Y_loc_train_n},
    validation_data=([Z_PRE_val, Z_PHYS_val], {"presence": Y_cls_val, "coords": Y_loc_val_n}),
    batch_size=64,
    epochs=50,
    callbacks=[ckpt_phase1, es_phase1],
    verbose=2,
)

print("✔ Phase 1 done.")

Phase 1: fallback freeze by name_contains='precnn'
Epoch 1/50


I0000 00:00:1768504147.264410 2341054 service.cc:152] XLA service 0x7ff958002bc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1768504147.264422 2341054 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2026-01-15 19:09:07.290184: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1768504147.455529 2341054 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1768504149.578553 2341054 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2026-01-15 19:09:14.517638: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5', 12 bytes spill stores, 12 bytes spill loads

2026-01-15 19:09:16.182663: I external/l


Epoch 1: val_coords_loss improved from inf to 0.43678, saving model to /home/tonyliao/Location_new_aoa_PDF/models/fine_phase1.h5


2077/2077 - 11s - 5ms/step - coords_euclid_mean: 0.7716 - coords_loss: 0.5109 - loss: 0.5168 - presence_accuracy: 0.9916 - presence_loss: 0.0294 - val_coords_euclid_mean: 0.6795 - val_coords_loss: 0.4368 - val_loss: 0.4368 - val_presence_accuracy: 1.0000 - val_presence_loss: 6.3131e-04
Epoch 2/50

Epoch 2: val_coords_loss improved from 0.43678 to 0.42372, saving model to /home/tonyliao/Location_new_aoa_PDF/models/fine_phase1.h5


2077/2077 - 3s - 2ms/step - coords_euclid_mean: 0.6739 - coords_loss: 0.4413 - loss: 0.4413 - presence_accuracy: 1.0000 - presence_loss: 2.6497e-04 - val_coords_euclid_mean: 0.6564 - val_coords_loss: 0.4237 - val_loss: 0.4237 - val_presence_accuracy: 1.0000 - val_presence_loss: 1.6154e-04
Epoch 3/50

Epoch 3: val_coords_loss did not improve from 0.42372
2077/2077 - 4s - 2ms/step - coords_euclid_mean: 0.6605 - coords_loss: 0.4338 - loss: 0.4339 - presence_accuracy: 1.0000 - presence_loss: 9.4205e-05 - val_coords_euclid_mean: 0.6501 - val_coords_loss: 0.4307 - val_loss: 0.4307 - val_presence_accuracy: 1.0000 - val_presence_loss: 4.6615e-05
Epoch 4/50

Epoch 4: val_coords_loss did not improve from 0.42372
2077/2077 - 5s - 2ms/step - coords_euclid_mean: 0.6512 - coords_loss: 0.4276 - loss: 0.4276 - presence_accuracy: 1.0000 - presence_loss: 3.4595e-05 - val_coords_euclid_mean: 0.6558 - val_coords_loss: 0.4250 - val_loss: 0.4249 - val_presence_accuracy: 1.0000 - val_presence_loss: 2.3569e-0

2077/2077 - 4s - 2ms/step - coords_euclid_mean: 0.6480 - coords_loss: 0.4257 - loss: 0.4257 - presence_accuracy: 1.0000 - presence_loss: 1.5181e-05 - val_coords_euclid_mean: 0.6476 - val_coords_loss: 0.4173 - val_loss: 0.4173 - val_presence_accuracy: 1.0000 - val_presence_loss: 8.9946e-06
Epoch 6/50

Epoch 6: val_coords_loss did not improve from 0.41730
2077/2077 - 4s - 2ms/step - coords_euclid_mean: 0.6446 - coords_loss: 0.4243 - loss: 0.4243 - presence_accuracy: 1.0000 - presence_loss: 6.0900e-06 - val_coords_euclid_mean: 0.6515 - val_coords_loss: 0.4206 - val_loss: 0.4205 - val_presence_accuracy: 1.0000 - val_presence_loss: 4.0710e-06
Epoch 7/50

Epoch 7: val_coords_loss did not improve from 0.41730
2077/2077 - 4s - 2ms/step - coords_euclid_mean: 0.6433 - coords_loss: 0.4232 - loss: 0.4233 - presence_accuracy: 1.0000 - presence_loss: 2.1309e-06 - val_coords_euclid_mean: 0.6474 - val_coords_loss: 0.4223 - val_loss: 0.4222 - val_presence_accuracy: 1.0000 - val_presence_loss: 1.6819e-0

2077/2077 - 5s - 2ms/step - coords_euclid_mean: 0.6353 - coords_loss: 0.4176 - loss: 0.4176 - presence_accuracy: 1.0000 - presence_loss: 1.3894e-07 - val_coords_euclid_mean: 0.6313 - val_coords_loss: 0.4121 - val_loss: 0.4120 - val_presence_accuracy: 1.0000 - val_presence_loss: 8.2594e-08
Epoch 11/50

Epoch 11: val_coords_loss did not improve from 0.41215
2077/2077 - 5s - 3ms/step - coords_euclid_mean: 0.6340 - coords_loss: 0.4177 - loss: 0.4177 - presence_accuracy: 1.0000 - presence_loss: 8.0556e-08 - val_coords_euclid_mean: 0.6321 - val_coords_loss: 0.4160 - val_loss: 0.4159 - val_presence_accuracy: 1.0000 - val_presence_loss: 4.7633e-08
Epoch 12/50

Epoch 12: val_coords_loss did not improve from 0.41215
2077/2077 - 4s - 2ms/step - coords_euclid_mean: 0.6318 - coords_loss: 0.4149 - loss: 0.4149 - presence_accuracy: 1.0000 - presence_loss: 4.6908e-08 - val_coords_euclid_mean: 0.6375 - val_coords_loss: 0.4151 - val_loss: 0.4151 - val_presence_accuracy: 1.0000 - val_presence_loss: 3.672

2077/2077 - 5s - 2ms/step - coords_euclid_mean: 0.6304 - coords_loss: 0.4150 - loss: 0.4150 - presence_accuracy: 1.0000 - presence_loss: 2.7527e-08 - val_coords_euclid_mean: 0.6284 - val_coords_loss: 0.4104 - val_loss: 0.4103 - val_presence_accuracy: 1.0000 - val_presence_loss: 9.4412e-09
Epoch 14/50

Epoch 14: val_coords_loss did not improve from 0.41044
2077/2077 - 4s - 2ms/step - coords_euclid_mean: 0.6311 - coords_loss: 0.4151 - loss: 0.4151 - presence_accuracy: 1.0000 - presence_loss: 1.2922e-08 - val_coords_euclid_mean: 0.6265 - val_coords_loss: 0.4106 - val_loss: 0.4105 - val_presence_accuracy: 1.0000 - val_presence_loss: 1.4500e-08
Epoch 15/50

Epoch 15: val_coords_loss did not improve from 0.41044
2077/2077 - 4s - 2ms/step - coords_euclid_mean: 0.6267 - coords_loss: 0.4116 - loss: 0.4116 - presence_accuracy: 1.0000 - presence_loss: 1.4686e-08 - val_coords_euclid_mean: 0.6353 - val_coords_loss: 0.4206 - val_loss: 0.4205 - val_presence_accuracy: 1.0000 - val_presence_loss: 1.181

In [5]:
# =========================================================
# CELL 5 — Phase 2: Unfreeze Top PreCNN Blocks + Compile + Train
# =========================================================
def unfreeze_top_blocks(precnn_model: tf.keras.Model, keep_frozen_ratio: float = 0.70):
    """
    Unfreeze only the last ~30% layers of the PreCNN submodel.
    This is more robust than matching conv1d layer names.
    """
    layers = precnn_model.layers
    cut = int(len(layers) * keep_frozen_ratio)
    for i, layer in enumerate(layers):
        layer.trainable = (i >= cut)

precnn = find_submodel(full_model, "precnn") or find_submodel(full_model, "PreCNN")
if precnn is not None:
    unfreeze_top_blocks(precnn, keep_frozen_ratio=0.70)
    print("Phase 2: unfroze top blocks in:", precnn.name)
else:
    # fallback: unfreeze conv/bn-ish layers by name if no submodel found
    for layer in full_model.layers:
        name = layer.name.lower()
        if ("conv" in name) or ("batch" in name) or ("bn" in name):
            layer.trainable = True
    print("Phase 2: fallback unfreeze conv/bn layers in full_model")

LR_PHASE2 = 1e-4
full_model.compile(
    optimizer=optimizers.Adam(LR_PHASE2),
    loss={
        "presence": losses.SparseCategoricalCrossentropy(),
        "coords":   losses.Huber(delta=0.5),
    },
    loss_weights={"presence": 0.2, "coords": 1.0},
    metrics={"presence": ["accuracy"], "coords": [euclid_mean]},
)

ckpt_phase2 = callbacks.ModelCheckpoint(
    f"{MODEL_DIR}/fine_phase2.h5",
    save_best_only=True,
    monitor="val_coords_euclid_mean",
    mode="min",
    verbose=1,
)
es_phase2 = callbacks.EarlyStopping(
    monitor="val_coords_euclid_mean",
    mode="min",
    patience=12,
    min_delta=1e-4,
    restore_best_weights=True,
    verbose=1,
)

history_phase2 = full_model.fit(
    x=[Z_PRE_train, Z_PHYS_train],
    y={"presence": Y_cls_train, "coords": Y_loc_train_n},
    validation_data=([Z_PRE_val, Z_PHYS_val], {"presence": Y_cls_val, "coords": Y_loc_val_n}),
    batch_size=64,
    epochs=20,
    callbacks=[ckpt_phase2, es_phase2],
    verbose=2,
)

print("✔ Phase 2 done.")


Phase 2: fallback unfreeze conv/bn layers in full_model
Epoch 1/20

Epoch 1: val_coords_euclid_mean improved from inf to 0.56499, saving model to /home/tonyliao/Location_new_aoa_PDF/models/fine_phase2.h5


2077/2077 - 8s - 4ms/step - coords_euclid_mean: 0.5711 - coords_loss: 0.1208 - loss: 0.1208 - presence_accuracy: 1.0000 - presence_loss: 7.3377e-09 - val_coords_euclid_mean: 0.5650 - val_coords_loss: 0.1205 - val_loss: 0.1205 - val_presence_accuracy: 1.0000 - val_presence_loss: 4.2698e-09
Epoch 2/20

Epoch 2: val_coords_euclid_mean improved from 0.56499 to 0.55502, saving model to /home/tonyliao/Location_new_aoa_PDF/models/fine_phase2.h5


2077/2077 - 4s - 2ms/step - coords_euclid_mean: 0.5530 - coords_loss: 0.1178 - loss: 0.1178 - presence_accuracy: 1.0000 - presence_loss: 5.1694e-09 - val_coords_euclid_mean: 0.5550 - val_coords_loss: 0.1189 - val_loss: 0.1189 - val_presence_accuracy: 1.0000 - val_presence_loss: 3.5784e-09
Epoch 3/20

Epoch 3: val_coords_euclid_mean improved from 0.55502 to 0.55259, saving model to /home/tonyliao/Location_new_aoa_PDF/models/fine_phase2.h5


2077/2077 - 4s - 2ms/step - coords_euclid_mean: 0.5472 - coords_loss: 0.1168 - loss: 0.1168 - presence_accuracy: 1.0000 - presence_loss: 3.9830e-09 - val_coords_euclid_mean: 0.5526 - val_coords_loss: 0.1187 - val_loss: 0.1187 - val_presence_accuracy: 1.0000 - val_presence_loss: 3.2202e-09
Epoch 4/20

Epoch 4: val_coords_euclid_mean improved from 0.55259 to 0.54989, saving model to /home/tonyliao/Location_new_aoa_PDF/models/fine_phase2.h5


2077/2077 - 5s - 2ms/step - coords_euclid_mean: 0.5449 - coords_loss: 0.1164 - loss: 0.1164 - presence_accuracy: 1.0000 - presence_loss: 3.9892e-09 - val_coords_euclid_mean: 0.5499 - val_coords_loss: 0.1183 - val_loss: 0.1183 - val_presence_accuracy: 1.0000 - val_presence_loss: 3.0232e-09
Epoch 5/20

Epoch 5: val_coords_euclid_mean did not improve from 0.54989
2077/2077 - 5s - 3ms/step - coords_euclid_mean: 0.5421 - coords_loss: 0.1157 - loss: 0.1157 - presence_accuracy: 1.0000 - presence_loss: 3.9432e-09 - val_coords_euclid_mean: 0.5517 - val_coords_loss: 0.1182 - val_loss: 0.1182 - val_presence_accuracy: 1.0000 - val_presence_loss: 2.7367e-09
Epoch 6/20

Epoch 6: val_coords_euclid_mean improved from 0.54989 to 0.54844, saving model to /home/tonyliao/Location_new_aoa_PDF/models/fine_phase2.h5


...

2077/2077 - 5s - 2ms/step - coords_euclid_mean: 0.5371 - coords_loss: 0.1147 - loss: 0.1147 - presence_accuracy: 1.0000 - presence_loss: 3.4643e-09 - val_coords_euclid_mean: 0.5475 - val_coords_loss: 0.1176 - val_loss: 0.1176 - val_presence_accuracy: 1.0000 - val_presence_loss: 2.8764e-09
Epoch 10/20

Epoch 10: val_coords_euclid_mean improved from 0.54750 to 0.54483, saving model to /home/tonyliao/Location_new_aoa_PDF/models/fine_phase2.h5


2077/2077 - 4s - 2ms/step - coords_euclid_mean: 0.5361 - coords_loss: 0.1144 - loss: 0.1144 - presence_accuracy: 1.0000 - presence_loss: 3.9783e-09 - val_coords_euclid_mean: 0.5448 - val_coords_loss: 0.1175 - val_loss: 0.1175 - val_presence_accuracy: 1.0000 - val_presence_loss: 2.6829e-09
Epoch 11/20

Epoch 11: val_coords_euclid_mean did not improve from 0.54483
2077/2077 - 5s - 3ms/step - coords_euclid_mean: 0.5349 - coords_loss: 0.1142 - loss: 0.1142 - presence_accuracy: 1.0000 - presence_loss: 3.5423e-09 - val_coords_euclid_mean: 0.5454 - val_coords_loss: 0.1175 - val_loss: 0.1175 - val_presence_accuracy: 1.0000 - val_presence_loss: 2.5826e-09
Epoch 12/20

Epoch 12: val_coords_euclid_mean did not improve from 0.54483
2077/2077 - 4s - 2ms/step - coords_euclid_mean: 0.5338 - coords_loss: 0.1140 - loss: 0.1140 - presence_accuracy: 1.0000 - presence_loss: 3.9251e-09 - val_coords_euclid_mean: 0.5476 - val_coords_loss: 0.1174 - val_loss: 0.1174 - val_presence_accuracy: 1.0000 - val_presen

2077/2077 - 5s - 2ms/step - coords_euclid_mean: 0.5312 - coords_loss: 0.1133 - loss: 0.1133 - presence_accuracy: 1.0000 - presence_loss: 2.9753e-09 - val_coords_euclid_mean: 0.5446 - val_coords_loss: 0.1174 - val_loss: 0.1173 - val_presence_accuracy: 1.0000 - val_presence_loss: 2.2173e-09
Epoch 16/20

Epoch 16: val_coords_euclid_mean improved from 0.54456 to 0.54286, saving model to /home/tonyliao/Location_new_aoa_PDF/models/fine_phase2.h5


2077/2077 - 4s - 2ms/step - coords_euclid_mean: 0.5313 - coords_loss: 0.1133 - loss: 0.1133 - presence_accuracy: 1.0000 - presence_loss: 3.0913e-09 - val_coords_euclid_mean: 0.5429 - val_coords_loss: 0.1172 - val_loss: 0.1172 - val_presence_accuracy: 1.0000 - val_presence_loss: 2.1277e-09
Epoch 17/20

Epoch 17: val_coords_euclid_mean did not improve from 0.54286
2077/2077 - 5s - 2ms/step - coords_euclid_mean: 0.5303 - coords_loss: 0.1131 - loss: 0.1131 - presence_accuracy: 1.0000 - presence_loss: 2.9920e-09 - val_coords_euclid_mean: 0.5449 - val_coords_loss: 0.1179 - val_loss: 0.1179 - val_presence_accuracy: 1.0000 - val_presence_loss: 2.1564e-09
Epoch 18/20

Epoch 18: val_coords_euclid_mean improved from 0.54286 to 0.54252, saving model to /home/tonyliao/Location_new_aoa_PDF/models/fine_phase2.h5


2077/2077 - 4s - 2ms/step - coords_euclid_mean: 0.5296 - coords_loss: 0.1129 - loss: 0.1129 - presence_accuracy: 1.0000 - presence_loss: 2.9981e-09 - val_coords_euclid_mean: 0.5425 - val_coords_loss: 0.1174 - val_loss: 0.1174 - val_presence_accuracy: 1.0000 - val_presence_loss: 2.3641e-09
Epoch 19/20

Epoch 19: val_coords_euclid_mean did not improve from 0.54252
2077/2077 - 4s - 2ms/step - coords_euclid_mean: 0.5284 - coords_loss: 0.1127 - loss: 0.1127 - presence_accuracy: 1.0000 - presence_loss: 3.2310e-09 - val_coords_euclid_mean: 0.5449 - val_coords_loss: 0.1176 - val_loss: 0.1175 - val_presence_accuracy: 1.0000 - val_presence_loss: 2.3856e-09
Epoch 20/20

Epoch 20: val_coords_euclid_mean did not improve from 0.54252
2077/2077 - 5s - 3ms/step - coords_euclid_mean: 0.5284 - coords_loss: 0.1127 - loss: 0.1127 - presence_accuracy: 1.0000 - presence_loss: 3.4045e-09 - val_coords_euclid_mean: 0.5464 - val_coords_loss: 0.1173 - val_loss: 0.1173 - val_presence_accuracy: 1.0000 - val_presen

In [6]:
# =========================================================
# CELL 6 — Save Final Model + Compute coord_bias.npy
# =========================================================
FINAL_SAVE = f"{MODEL_DIR}/final_finetuned_model.h5"
full_model.save(FINAL_SAVE)
print("Final model saved →", FINAL_SAVE)

# ---- compute coord_bias (Eq.8-style systematic offset correction) ----
pred = full_model.predict([Z_PRE_val, Z_PHYS_val], verbose=0)

def extract_coords_pred(pred_out):
    # pred_out can be list/tuple or dict
    if isinstance(pred_out, dict):
        return pred_out["coords"]
    if isinstance(pred_out, (list, tuple)):
        # pick the (N,2) array
        for arr in pred_out:
            if isinstance(arr, np.ndarray) and arr.ndim == 2 and arr.shape[1] == 2:
                return arr
    raise ValueError("Cannot find coords output in model.predict() result")

pred_loc_n = extract_coords_pred(pred).astype(np.float32)          # normalized coords
pred_loc   = pred_loc_n * coord_std + coord_mean                   # real coords

# Exclude "empty" samples if your Y_cls uses 0 as empty (common)
EMPTY_CLASS_ID = 0
mask = (Y_cls_val != EMPTY_CLASS_ID)
if mask.sum() < 10:
    # if dataset doesn't use 0-as-empty or not enough samples, fall back to all
    mask = np.ones(len(Y_cls_val), dtype=bool)

coord_bias = (pred_loc[mask] - Y_loc_val[mask]).mean(axis=0).astype(np.float32)
np.save(f"{MODEL_DIR}/coord_bias.npy", coord_bias)

print("Saved coord_bias →", f"{MODEL_DIR}/coord_bias.npy")
print("coord_bias:", coord_bias)
print("bias computed on samples:", int(mask.sum()), "/", len(mask))


Final model saved → /home/tonyliao/Location_new_aoa_PDF/models/final_finetuned_model.h5


2026-01-15 19:12:27.983603: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2', 4 bytes spill stores, 4 bytes spill loads

2026-01-15 19:12:28.048953: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_168', 4 bytes spill stores, 4 bytes spill loads

2026-01-15 19:12:28.195355: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_168', 12 bytes spill stores, 12 bytes spill loads



Saved coord_bias → /home/tonyliao/Location_new_aoa_PDF/models/coord_bias.npy
coord_bias: [0.00610671 1.6157156 ]
bias computed on samples: 33230 / 33230


In [7]:
# =========================================================
# CELL 7 — Optional quick eval + cleanup
# =========================================================
val_eval = full_model.evaluate(
    x=[Z_PRE_val, Z_PHYS_val],
    y={"presence": Y_cls_val, "coords": Y_loc_val_n},
    verbose=2,
)
print("Validation Evaluation:", val_eval)

tf.keras.backend.clear_session()

1039/1039 - 1s - 1ms/step - coords_euclid_mean: 0.5424 - coords_loss: 0.1174 - loss: 0.1174 - presence_accuracy: 1.0000 - presence_loss: 2.3664e-09
Validation Evaluation: [0.11738386750221252, 2.3664017412272642e-09, 0.11739612370729446, 0.5424419045448303, 1.0]


In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()
#Restart the kernel to free memory
import IPython
app = IPython.get_ipython()
app.kernel.do_shutdown(True)  # True = restart, False = shutdown

{'status': 'ok', 'restart': True}

: 